To run this notebook, you need:
* [fabmos](https://github.com/BoldingBruggeman/fabmos/wiki)
* [matplotlib](https://matplotlib.org/)
* [cartopy](https://scitools.org.uk/cartopy/)
* [cmocean](https://matplotlib.org/cmocean/)
* [ipympl](https://matplotlib.org/ipympl/)

To install these with anaconda:

```conda install -c conda-forge fabmos matplotlib cartopy cmocean ipympl```

In [1]:
import numpy as np
import netCDF4
%matplotlib widget
from matplotlib import pyplot
import ipywidgets
import fabmos.input.cluster
import run

import cmocean
import datetime
import cartopy.crs as ccrs
from cartopy.feature import LAND

In [15]:
REF_FILE = "SOM/daily_surface_climatology_2007_2016.nc"
EXP_FILE = "gotm-arctic.nc"
PLOT_TEMPERATURE = True

if PLOT_TEMPERATURE:
    REF_NAME, EXP_NAME = "temp", "temp"
    vmin_fixed, vmax_fixed = 0.0, 12.0
    cmap = cmocean.cm.thermal
    LOGSCALE = False
else:
    # REF_NAME, EXP_NAME = "ECO_diac", "ECO_diachl"
    REF_NAME, EXP_NAME = "ECO_flac", "ECO_flachl"
    vmin_fixed, vmax_fixed = 0.0, 5.0
    cmap = cmocean.cm.algae
    LOGSCALE = True

# Clusters

In [ ]:
domain = run.get_domain(cluster=True)
cluster_index = np.ma.masked_where(domain.full_mask == 0, domain.full_cluster_index)
cluster_ids = domain.cluster_ids.astype(int)

fig, ax = pyplot.subplots(
    figsize=(6, 6), subplot_kw=dict(projection=ccrs.NorthPolarStereo())
)
ax.pcolormesh(
    domain.full_lon,
    domain.full_lat,
    cluster_index,
    cmap="tab20",
    vmin=0,
    vmax=19,
    shading="auto",
    transform=ccrs.PlateCarree(),
)
ax.coastlines("50m", lw=0.5)
ax.gridlines(
    draw_labels=True,
    dms=True,
    x_inline=False,
    y_inline=False,
    rotate_labels=False,
    color="k",
    ls=":",
)
ax.add_feature(LAND.with_scale("50m"), fc=[0.9, 0.9, 0.9])
ax.set_extent(
    [domain.full_lon.min(), domain.full_lon.max(), 51, domain.full_lat.max()],
    crs=ccrs.PlateCarree(),
)

for icluster, i, j in fabmos.input.cluster.get_representative_points(
    cluster_index, mincount=20
):
    ax.text(
        domain.full_lon[j, i],
        domain.full_lat[j, i],
        cluster_ids[icluster],
        horizontalalignment="center",
        verticalalignment="center",
        transform=ccrs.PlateCarree(),
    )
fig.savefig("clusters.png", dpi=300)

# Connectivity

In [ ]:
# Load cluster connectivity = time-averaged flow between clusters
with netCDF4.Dataset("connections.nc") as nc:
    source2target = nc["exchange"][:, :, :].mean(axis=0)
np.fill_diagonal(source2target, 0.0)
h_inc = source2target.sum(axis=1) * 365 * 86400
h_dec = source2target.sum(axis=0) * 365 * 86400

vol = domain.area[1, 1::2] * domain.H[1, 1::2]
fig, (ax, ax1) = pyplot.subplots(figsize=(10, 4), ncols=2)
ax.bar(np.arange(h_inc.size) - 0.2, h_inc / vol, width=0.35, label="inflow")
ax.bar(np.arange(h_inc.size) + 0.2, h_dec / vol, width=0.35, label="outflow")
ax.grid()
ax.set_xlabel("cluster")
ax.set_ylabel("flow relative to cluster volume (year-1)")
ax.xaxis.set_ticks(range(h_inc.size), [str(v) for v in cluster_ids])
ax.legend()

i = np.arange(h_inc.size)
mask = np.zeros_like(source2target, dtype=bool)
mask[i, i] = True
mask |= source2target == 0.0
source2target = np.ma.array(source2target, mask=mask)
pc = ax1.pcolormesh(source2target)
cb = fig.colorbar(pc, ax=ax1)
cb.set_label("mean flow (m3 s-1)")
ax1.xaxis.set_ticks(np.arange(h_inc.size) + 0.5, [str(v) for v in cluster_ids])
ax1.yaxis.set_ticks(np.arange(h_inc.size) + 0.5, [str(v) for v in cluster_ids])
ax.set_title("relative inflow and outflow")
ax1.set_xlabel("source cluster")
ax1.set_ylabel("destination cluster")
ax1.set_title("flow between clusters")
fig.savefig("exchange.png", dpi=300)

# Compare maps

Cluster simulation vs. original CMEMS results

In [ ]:
IYEAR = 0

with netCDF4.Dataset(EXP_FILE) as nc:
    y, x = np.indices(cluster_index.shape)
    cluster_index = nc["cluster_index"][:, :]
    times = netCDF4.num2date(nc["time"], nc["time"].units)

fig, ((ax0, ax1, cax),) = pyplot.subplots(
    ncols=3, width_ratios=(0.45, 0.45, 0.1), figsize=(8, 4), squeeze=False
)

exp_offset = IYEAR * 365

def plot(iday=-1):
    with netCDF4.Dataset(REF_FILE) as nc:
        values_ref = nc[REF_NAME][iday, 0, :, :]
        values_ref = np.ma.array(values_ref, mask=cluster_index.mask)

    with netCDF4.Dataset(EXP_FILE) as nc:
        values_exp = nc[EXP_NAME][iday + exp_offset, -1, 0, :]

    vmin = (
        vmin_fixed
        if vmin_fixed is not None
        else min(values_ref.min(), values_exp.min())
    )
    vmax = (
        vmax_fixed
        if vmax_fixed is not None
        else max(values_ref.max(), values_exp.max())
    )

    ax0.cla()
    ax1.cla()
    cax.cla()

    pc = ax0.pcolormesh(x, y, values_ref, vmin=vmin, vmax=vmax, cmap=cmap)
    values_exp_unpacked = np.ma.array(
        values_exp[cluster_index], mask=cluster_index.mask
    )
    pc = ax1.pcolormesh(x, y, values_exp_unpacked, vmin=vmin, vmax=vmax, cmap=cmap)
    ax1.contour(
        x,
        y,
        cluster_index,
        np.arange(cluster_index.max() + 1),
        colors="k",
        linewidths=0.2,
    )

    fig.colorbar(pc, cax=cax)
    ax0.xaxis.set_ticks(())
    ax0.yaxis.set_ticks(())
    ax1.xaxis.set_ticks(())
    ax1.yaxis.set_ticks(())
    cb = fig.colorbar(pc, cax=cax)
    cb.set_label(REF_NAME)

ipywidgets.interact(plot, iday=(0, 364));

# Compare timeseries

In [ ]:
fig_ts, (ax_ts, ax_cluster) = pyplot.subplots(ncols=2, figsize=(8, 4))

with netCDF4.Dataset(EXP_FILE) as nc:
    cluster_x = nc["icluster"][:]
    cluster_y = nc["jcluster"][:]
    values_exp = nc[EXP_NAME][slice(-365, None), -1, 0, :]
times = np.arange(values_exp.shape[0])
if REF_FILE is not None:
    with netCDF4.Dataset(REF_FILE) as nc:
        values_ref = nc[REF_NAME][:, 0, :, :]

def plot_timeseries(icluster):
    ax_ts.cla()
    mask = cluster_index == icluster
    ncells = mask.sum()
    if REF_FILE is not None:
        cluster_values_ref = np.asarray(values_ref[:, mask])
        (perc25, median, perc75) = np.percentile(
            cluster_values_ref, (25, 50, 75), axis=-1
        )
        min_ref = cluster_values_ref.min(axis=-1)
        max_ref = cluster_values_ref.max(axis=-1)
        if ncells > 1:
            ax_ts.fill_between(
                times, min_ref, max_ref, fc=[0.8, 0.8, 0.8], label="original min-max"
            )
            ax_ts.fill_between(
                times, perc25, perc75, fc=[0.6, 0.6, 0.6], label="original 25-75%"
            )
        ax_ts.plot(times, median, "--k", label="original median")
    ax_ts.plot(times, values_exp[:, icluster], "-k", label="clustered")
    ax_ts.set_title(f"cluster {icluster}, {ncells} cells")
    ax_ts.grid()
    ax_ts.legend()
    ax_cluster.cla()
    ax_cluster.pcolormesh(mask)

ipywidgets.interact(plot_timeseries, icluster=(0, cluster_x.size - 1))

# Compare all monthly maps

In [ ]:
# Compare results from cluster simulation with original CMEMS results
with netCDF4.Dataset(EXP_FILE) as nc:
    cluster_index = nc["cluster_index"][:, :]
    y, x = np.indices(cluster_index.shape)
    times = netCDF4.num2date(nc["time"], nc["time"].units)

y = domain.full_lat
x = domain.full_lon

exp_offset = 0  # 0 #365 - len(times)


def plot(month, ax0, ax1, itime=-1):
    with netCDF4.Dataset(REF_FILE) as nc:
        values_ref = nc[REF_NAME][itime, 0, :, :]
        # values_ref = nc[REF_NAME][0, :, :]  # this for REF_FILE==glodap_ip.nc
        values_ref = np.ma.array(values_ref, mask=cluster_index.mask)
    with netCDF4.Dataset(EXP_FILE) as nc:
        values_exp = nc[EXP_NAME][itime + exp_offset, -1, 0, :]
    vmin = (
        vmin_fixed
        if vmin_fixed is not None
        else min(values_ref.min(), values_exp.min())
    )
    vmax = (
        vmax_fixed
        if vmax_fixed is not None
        else max(values_ref.max(), values_exp.max())
    )
    pc = ax0.pcolormesh(
        x, y, values_ref, vmin=vmin, vmax=vmax, cmap=cmap, transform=ccrs.PlateCarree()
    )
    values_exp_unpacked = np.ma.array(
        values_exp[cluster_index], mask=cluster_index.mask
    )
    pc = ax1.pcolormesh(
        x,
        y,
        values_exp_unpacked,
        vmin=vmin,
        vmax=vmax,
        cmap=cmap,
        transform=ccrs.PlateCarree(),
    )
    # ax1.contour(x, y, cluster_index, np.arange(cluster_index.max() + 1), colors='k', linewidths=0.2, transform=ccrs.PlateCarree())
    # fig.colorbar(pc, cax=cax)
    ax0.xaxis.set_ticks(())
    ax0.yaxis.set_ticks(())
    ax0.axis("off")
    ax1.xaxis.set_ticks(())
    ax1.yaxis.set_ticks(())
    ax1.axis("off")
    # cb = fig.colorbar(pc, cax=cax)
    # cb.set_label(REF_NAME)

    label = datetime.datetime(2000, month, 1).strftime("%b")
    ax0.text(
        0.95,
        0.9,
        label,
        transform=ax0.transAxes,
        verticalalignment="center",
        horizontalalignment="center",
    )


fig, axes = pyplot.subplots(
    nrows=6,
    ncols=4,
    figsize=(8, 16),
    subplot_kw=dict(projection=ccrs.NorthPolarStereo()),
    gridspec_kw=dict(wspace=-0.2, hspace=0),
)
for i in range(12):
    irow = i // 2
    icol = 2 * (i % 2)
    plot(i + 1, axes[irow, icol], axes[irow, icol + 1], i * 30 + 15)
axes[0, 0].text(
    0.5, 1.2, "original", transform=axes[0, 0].transAxes, horizontalalignment="center"
)
axes[0, 1].text(
    0.5, 1.2, "clustered", transform=axes[0, 1].transAxes, horizontalalignment="center"
)
axes[0, 0].text(
    0.5, 1.2, "original", transform=axes[0, 2].transAxes, horizontalalignment="center"
)
axes[0, 1].text(
    0.5, 1.2, "clustered", transform=axes[0, 3].transAxes, horizontalalignment="center"
)
fig.savefig(f"{REF_NAME}.png", dpi=300)

# Compare timeseries per cluster

In [ ]:
# Compare results from cluster simulation with original CMEMS results
with netCDF4.Dataset(EXP_FILE) as nc:
    cluster_index = nc["cluster_index"][:, :]
    y, x = np.indices(cluster_index.shape)
    times = netCDF4.num2date(nc["time"], nc["time"].units)

y = domain.full_lat
x = domain.full_lon

exp_offset = 0  # 0 #365 - len(times)


def get_clim(values, n=365):
    res = np.zeros((n,), values.dtype)
    ncopies = values.size // n
    for i in range(ncopies):
        res += values[i * 365 : (i + 1) * 365]
    return res / ncopies


def plot_timeseries(icluster, ax):
    with netCDF4.Dataset(REF_FILE) as nc:
        values_ref = nc[REF_NAME][:, 0, :, :]
        cluster_values_ref = values_ref[:, cluster_index == icluster]
        (perc25, median, perc75) = np.percentile(
            cluster_values_ref, (25, 50, 75), axis=-1
        )
        values_ref = cluster_values_ref.mean(axis=-1)
        values_ref_min = cluster_values_ref.min(axis=-1)
        values_ref_max = cluster_values_ref.max(axis=-1)
    with netCDF4.Dataset(EXP_FILE) as nc:
        values_exp = nc[EXP_NAME][:, -1, 0, icluster]
        values_exp = get_clim(values_exp)

    vmin = (
        vmin_fixed
        if vmin_fixed is not None
        else min(values_ref.min(), values_exp.min())
    )
    vmax = (
        vmax_fixed
        if vmax_fixed is not None
        else max(values_ref.max(), values_exp.max())
    )

    times = np.arange(365)
    ax.fill_between(times, values_ref_min, values_ref_max, fc="k", alpha=0.25)
    ax.fill_between(times, perc25, perc75, fc="k", alpha=0.25)
    ax.plot(times, median, "-k", label="original")
    ax.plot(values_exp, label="cluster")
    ax.grid()
    ax.set_title(cluster_ids[icluster])
    ax.legend()
    ax.set_ylim(vmin - 2, vmax + 2)
    if LOGSCALE:
        ax.set_yscale("log")
        ax.set_ylim(0.01, 10.0)


fig, axes = pyplot.subplots(
    nrows=6,
    ncols=2,
    figsize=(8, 16),
)
for i in range(12):
    plot_timeseries(i, axes.ravel()[i])